In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import os 
import requests

In [6]:
def download_document(file_name, document_url):
    if os.path.exists(file_name):
        pass
    else:
        response = requests.get(document_url)
        if response.status_code == 200:
            with open(file_name, 'wb') as f:
                f.write(response.content)
            return f    

        else:
            print(f'Failed to download the document. Status code: {response.status_code}')



file_name = 'movies.csv'
document_url = ' https://gist.githubusercontent.com/tiangechen/b68782efa49a16edaf07dc2cdaa855ea/raw/0c794a9717f18b094eabab2cd6a6b9a226903577'
download_document(file_name, document_url)

In [5]:
df = pd.read_csv(file_name)
df.drop_duplicates(inplace = True)

In [6]:
df.head()

,Film,Genre,Lead Studio,Audience score %,Profitability,Rotten Tomatoes %,Worldwide Gross,Year
0,Zack and Miri Make a Porno,Romance,The Weinstein Company,70,1.747542,64,$41.94,2008
1,Youth in Revolt,Comedy,The Weinstein Company,52,1.090000,68,$19.62,2010
2,You Will Meet a Tall Dark Stranger,Comedy,Independent,35,1.211818,43,$26.66,2010
3,When in Rome,Comedy,Disney,44,0.000000,15,$43.04,2010
4,What Happens in Vegas,Comedy,Fox,72,6.267647,28,$219.37,2008


In [9]:
df['Worldwide Gross'] = df['Worldwide Gross'].str[1:]# прибираю знак $
df = df.astype({'Worldwide Gross': 'float64'}) # змінюю тип даних для поля Worldwide Gross - воно має бути числовим для аналізу

### 2. Перерахуйте всі стовпці набору даних та вивчіть їх типи. Вивчіть статистику з різних областей. Опишіть, які дані ми маємо

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 76
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Film               75 non-null     object 
 1   Genre              75 non-null     object 
 2   Lead Studio        75 non-null     object 
 3   Audience score %   75 non-null     int64  
 4   Profitability      75 non-null     float64
 5   Rotten Tomatoes %  75 non-null     int64  
 6   Worldwide Gross    75 non-null     float64
 7   Year               75 non-null     int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 5.3+ KB


In [11]:
round(df.describe(),2) # Numeric features

,Audience score %,Profitability,Rotten Tomatoes %,Worldwide Gross,Year
count,75.00,75.00,75.00,75.00,75.00
mean,63.72,4.53,46.49,138.38,2009.07
std,13.70,8.12,26.41,156.99,1.35
min,35.00,0.00,3.00,0.03,2007.00
25%,52.00,1.75,26.00,32.50,2008.00
50%,64.00,2.64,43.00,77.09,2009.00
75%,75.00,4.54,64.50,199.64,2010.00
max,89.00,66.93,96.00,709.82,2011.00


In [12]:
df.agg({'Film': 'nunique', 'Genre': 'nunique', 'Lead Studio': 'nunique'}) # Dimentions

Film           75
Genre          10
Lead Studio    13
dtype: int64

### 3.Скільки всього фільмів у наборі даних?

In [13]:
df['Film'].nunique()

75

### 4. Скільки фільмів є у наборі даних для кожного року?

In [14]:
df4 = df.groupby('Year')['Film'].count().reset_index()
df4

,Year,Film
0,2007,11
1,2008,19
2,2009,12
3,2010,20
4,2011,13


###  5. Покажіть детальну інформацію про найменш і найприбутковіші фільми в наборі

In [15]:
max_scores = df.loc[df['Profitability'].idxmax()]
min_scores = df.loc[df['Profitability'].idxmin()]

In [16]:
max_scores,min_scores

(Film                   Fireproof
 Genre                      Drama
 Lead Studio          Independent
 Audience score %              51
 Profitability             66.934
 Rotten Tomatoes %             40
 Worldwide Gross            33.47
 Year                        2008
 Name: 68, dtype: object,
 Film                 When in Rome
 Genre                      Comedy
 Lead Studio                Disney
 Audience score %               44
 Profitability                 0.0
 Rotten Tomatoes %              15
 Worldwide Gross             43.04
 Year                         2010
 Name: 3, dtype: object)

In [ ]:
print("The most profitable film:\n")

print(movies[movies['Profitability'] == movies['Profitability'].max()])

print("The least profitable film:\n")

print(movies[movies['Profitability'] == movies['Profitability'].min()])

# 2

max_prof = movies.Profitability.max()

min_prof = movies.Profitability.min()

print("The most profitable film:", movies.loc[movies['Profitability']. isin([max_prof])])

print("The least profitable film:", movies.loc[movies['Profitability']. isin([min_prof])])

### 6. Значення "Жанр" іноді здається непослідовним; спробуйте знайти  ці невідповідностей та виправити їх.

In [17]:
df['Genre'].unique()

array(['Romance', 'Comedy', 'Drama', 'Animation', 'Fantasy', 'Romence',
       'Comdy', 'Action', 'romance', 'comedy'], dtype=object)

In [18]:
df['Genre'] = df['Genre'].replace(['Comdy','comedy'],'Comedy').replace(['romance', 'Romence'],'Romance')

In [19]:
df['Genre'].unique()

array(['Romance', 'Comedy', 'Drama', 'Animation', 'Fantasy', 'Action'],
      dtype=object)

### 7. Збережіть (у новий файл CSV) 10 найкращих комедій за кількістю глядачів; покажіть лише назву фільму, рік та студію

In [8]:
df7 = df[df['Genre']=='Comedy'].sort_values(by=['Audience score %'],ascending = False).head(10)
df7.to_csv('HW6_task7.csv', index=False, columns = {'Film','Year','Lead Studio'})